Change Run-Time to Gpu

In [1]:
!pip install git+https://github.com/openai/whisper.git
!pip install transformers
!pip3 install imageio==2.4.1
!pip install moviepy

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-prx9p53o
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-prx9p53o
  Resolved https://github.com/openai/whisper.git to commit eff383b27b783e280c089475852ba83f20f64998
  Preparing metadata (setup.py) ... done
  Created wheel for whisper: filename=whisper-1.0-py3-none-any.whl size=1175240 sha256=a17e237838c5b0c1aa72d854acf93f019303825f72c02e740a09c60e254d9a18
  Stored in directory: /tmp/pip-ephem-wheel-cache-yicuw9sj/wheels/16/15/89/1c7bb31bd0006793a95549d04785121a8a36daad9158e1e43a
Successfully built whisper
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 19.2 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for imageio: filename=imageio-2.4.1-py3-none-any.whl size=3303885 sha256=e5b8828733f17b8f33c98e200f21e592907ef61fefdff08f4ac01ebed817df0b
  Stored in directory: /root/.cache/pip/wheels/46/20/07/7bb9c8c44e6ec2e

In [2]:
# from transformers import pipeline
# summarizer=pipeline("summarization", model="facebook/bart-large-cnn")


In [13]:
from moviepy.editor import *

import os
import math 
import pandas as pd
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
def run_range(duration):
  time=duration/30
  floor=math.ceil(time)
  return floor

time_range=30
clip_run_range=0
clip_duration=0

def audio_generator(path,aud=0,vid=0):
  global clip_run_range
  global clip_duration
  if vid==1:
    clip=VideoFileClip(path)
    clip_duration = clip.duration
    clip_run_range=run_range(clip_duration)
    for i in range(clip_run_range):
      left=i*time_range
      right=left+time_range
      # print(left,right)

      crop_clip=clip.subclip(left,right)
      try:
        crop_clip.audio.write_audiofile("vid_to_aud"+str(i)+".mp3")
      except:
        pass
  
  if aud==1:
    audio_clip=AudioFileClip(path)
    clip_duration = audio_clip.duration
    print(clip_duration)
    clip_run_range=run_range(clip_duration)
    print(clip_run_range)
    for i in range(clip_run_range):
      left=i*time_range
      right=left+time_range
      # print(left,right)
      crop_clip=audio_clip.subclip(left,right)
      try:
        crop_clip.write_audiofile("vid_to_aud"+str(i)+".mp3")
      except:
        pass

In [5]:
import whisper

model = whisper.load_model("large")



100%|██████████████████████████████████████| 2.87G/2.87G [00:16<00:00, 184MiB/s]


In [6]:

tokenizer = AutoTokenizer.from_pretrained("fabiochiu/t5-small-medium-title-generation")
title_model = AutoModelForSeq2SeqLM.from_pretrained("fabiochiu/t5-small-medium-title-generation")


Downloading:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/231M [00:00<?, ?B/s]

In [12]:
# from IPython.display import Audio
# Audio("/content/analysis.mp3")

In [7]:
# load audio and pad/trim it to fit 30 seconds
def SpeechToText(path,title_flag=1):
  audio = whisper.load_audio(path)
  audio = whisper.pad_or_trim(audio)

  # make log-Mel spectrogram and move to the same device as the model
  mel = whisper.log_mel_spectrogram(audio).to(model.device)

  # detect the spoken language
  _, probs = model.detect_language(mel)
  # print(f"Detected language: {max(probs, key=probs.get)}")

  # decode the audio
  dectected_lang=max(probs, key=probs.get)
  # print(dectected_lang)
  # print(type(dectected_lang))
  if dectected_lang != "en":
    options = whisper.DecodingOptions(task="translate",language="en")
    # print(options)
    translated = whisper.decode(model, mel, options)

  options2 = whisper.DecodingOptions(task="transcribe")
  # print(options2)
  transcribed = whisper.decode(model, mel, options2)

  # print the recognized text
  
  if title_flag==1:
    if dectected_lang != "en":
      text=translated.text
    else:
      text=transcribed.text

    inputs = ["summarize: " + text]

    inputs = tokenizer(inputs,truncation=True, return_tensors="pt")
    output = title_model.generate(**inputs,num_beams=2, do_sample=True, min_length=5, max_length=10)
    decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
    predicted_title = nltk.sent_tokenize(decoded_output.strip())[0]
    return translated.text,transcribed.text,predicted_title
  return translated.text,transcribed.text

In [11]:
audio_generator("../input/video-check/diabetes.mp4",vid=1)
transcribed_lit=[]
label_lit=[]
translated_lit=[]

MoviePy - Writing audio in vid_to_aud0.mp3


MoviePy - Done.
MoviePy - Writing audio in vid_to_aud1.mp3


MoviePy - Done.
MoviePy - Writing audio in vid_to_aud2.mp3


MoviePy - Done.
MoviePy - Writing audio in vid_to_aud3.mp3


MoviePy - Done.
MoviePy - Writing audio in vid_to_aud4.mp3


MoviePy - Done.


In [14]:
for i in tqdm(range(clip_run_range)):
    transcribed,label=SpeechToTextEng("./vid_to_aud"+str(i)+".mp3",1)
    transcribed_lit.append(transcribed)
    label_lit.append(label)
    os.remove("./vid_to_aud"+str(i)+".mp3")

100%|██████████| 5/5 [00:35<00:00,  7.09s/it]


In [16]:
# transcribed_lit
label_lit

['2 Types of Diabetes: Type 1 and',
 'Getting Type 1 Diabetes',
 'Diabetes symptoms, like having to wee',
 'Diabetes complications for people who suffer from slow healing',
 'Diabetes UK vs. Diabetes UK']

In [10]:
def SpeechToTextEng(path,title_flag=1):
  audio = whisper.load_audio(path)
  audio = whisper.pad_or_trim(audio)

  # make log-Mel spectrogram and move to the same device as the model
  mel = whisper.log_mel_spectrogram(audio).to(model.device)

  # detect the spoken language
  _, probs = model.detect_language(mel)
  # print(f"Detected language: {max(probs, key=probs.get)}")

  # decode the audio
  dectected_lang=max(probs, key=probs.get)
  # print(dectected_lang)
  # print(type(dectected_lang))
  # if dectected_lang != "en":
  #   options = whisper.DecodingOptions(task="translate",language="en")
  #   # print(options)
  #   translated = whisper.decode(model, mel, options)

  options2 = whisper.DecodingOptions(task="transcribe")
  # print(options2)
  transcribed = whisper.decode(model, mel, options2)
  # print(transcribed.text)
  # print the recognized text
  
  if title_flag==1:
    
    # if dectected_lang != "en":
    #   text=translated.text
    # else:
    #   text=transcribed.text
    text=transcribed.text
#     predicted_title=summarizer(text,max_length=10, min_length=0, do_sample=False)[0].get("summary_text")
    # old attempt
    inputs = ["summarize: " + text]

    inputs = tokenizer(inputs,truncation=True, return_tensors="pt")
    output = title_model.generate(**inputs, num_beams=2, do_sample=True, min_length=5, max_length=10)

    decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
    predicted_title = nltk.sent_tokenize(decoded_output.strip())[0]

    return transcribed.text,predicted_title
    
  return transcribed.text

In [70]:
audio_generator("../input/magic-of-maths/magic.m4a",aud=1)
transcribed_lit=[]
label_lit=[]
for i in range(clip_run_range):
    transcribed,label=SpeechToTextEng("./vid_to_aud"+str(i)+".mp3",1)
    transcribed_lit.append(transcribed)
    label_lit.append(label)
    os.remove("./vid_to_aud"+str(i)+".mp3")

In [71]:
print("transcribed - ",transcribed_lit)
print("label - ",label_lit)

transcribed -  ["I want to share the most beautiful equation in math with you, okay? I'm ready. So actually this is literally, if you search Google, the most beautiful equation in math, you will find it. So this thing I might need to use a little bit of the board. I'll just write this pi, we already know what pi is, right? There's another number in math which is called i. This is the square root of minus 1, so somehow i is equal to the square root of minus 1,", "there's no way you can multiply two numbers together and get a negative one. Well, there is if you allow what are called complex numbers or imaginary numbers. So that's this i. You could multiply those two together. You can get an even more crazy number. Not only is something that doesn't end, but it's not even a real number. But there's another number in math also. And that number is called e. And e appears on a lot of these scientific calculators. e is another number which is just like pi in the sense that it's transcendental

In [72]:
data = pd.DataFrame(
    {'transcriptions': transcribed_lit,
     'labels': label_lit
    })
data

,transcriptions,labels
0,I want to share the most beautiful equation in...,"The most beautiful equation in math with you,"
1,there's no way you can multiply two numbers to...,There's no way you can multiply two
2,it is. So this one goes by 2.71828 and so on. ...,This number is 2.71828 and so
3,"with 100 chocolates, and I dropped them all on...",3 crazy numbers.
4,and 0. And this is true. So what's going on he...,3 ridiculous numbers and 0
5,is actually the most beautiful equation in mat...,The shirts with the pi and the
6,"But actually, that should have one more punchl...","The last punchline should be, let me"
7,So of course that's negative one right there. ...,The t-shirts that are all going


In [73]:
data.to_csv('data.csv')